In [1]:
from darter.file import parse_elf_snapshot, parse_appjit_snapshot

# Loading and parsing the snapshot file

Here we open the file to inspect. It actually contains *two* snapshots, one is the common **base** and the other contains the actual **user code**.  
`parse_elf_snapshot` extracts the 2 blobs for each of the two snapshots, and parses them.
It only returns the second snapshot, which is the interesting one (and contains the base, too).

By default we are inspecting an included sample file which results from building the default Flutter app:

In [2]:
s = parse_elf_snapshot('samples/arm-app.so')

------- PARSING VM SNAPSHOT --------

[Header]
  length = 4733
  kind = 2 ('kFullAOT', 'Full + AOT code')

[Snapshot header]
  version = 'c8562f0ee0ebc38ba217c7955956d1cb'
  features = {'product': True, 'use_bare_instructions': True, 'asserts"': False, 'causal_async_stacks': True, 'bytecode': False, 'arm-eabi': True, 'softfp': True}

  base objects: 95
  objects: 935
  clusters: 5
  code order length = 69

[002c1094]: INFO: Reading allocation clusters...
[002c13a9]: INFO: Reading fill clusters...
[002c2215]: INFO: Reading roots...
[002c2281]: INFO: Snapshot parsed.

------- PARSING ISOLATE SNAPSHOT --------

[Header]
  length = 836159
  kind = 2 ('kFullAOT', 'Full + AOT code')

[Snapshot header]
  version = 'c8562f0ee0ebc38ba217c7955956d1cb'
  features = {'product': True, 'use_bare_instructions': True, 'asserts"': False, 'causal_async_stacks': True, 'bytecode': False, 'arm-eabi': True, 'softfp': True}

  base objects: 935
  objects: 74247
  clusters: 222
  code order length = 7228

[00

If your snapshot is AppJIT instead of AppAOT, you can use `parse_appjit_snapshot`:

In [3]:
s = parse_appjit_snapshot('samples/appjit.snapshot')

Blob lengths: (0, 0, 17599488, 6822944)
No base snapshot, skipping base snapshot parsing...

------- PARSING ISOLATE SNAPSHOT --------

[Header]
  length = 12708502
  kind = 1 ('kFullJIT', 'Full + JIT code')

[Snapshot header]
  version = 'c8562f0ee0ebc38ba217c7955956d1cb'
  features = {'release': True, 'use_bare_instructions': True, 'asserts"': False, 'use_field_guards"': True, 'use_osr"': True, 'causal_async_stacks': True, 'bytecode': False, 'x64-sysv': True}

  base objects: 934
  objects: 297885
  clusters: 160
  code order length = 0

[000010ad]: NOTICE: Snapshot expected 934 base objects, but the provided base has 95
[000010ad]: INFO: Reading allocation clusters...
[0003c3f2]: INFO: Reading fill clusters...
[00c1f990]: INFO: Reading roots...
[00c1fa9a]: INFO: Snapshot parsed.


If the parsing was successful, then you are good to go!

# Navigating the snapshot data

A Dart snapshot is a collection of objects. Each object is assigned a unique **ref number** starting with 1 (ref 0 is invalid).  
To access an object by its ref number, use `refs`:

In [5]:
s.refs[2436]

Class('package:myapp/main.dart', 'MyApp')->2436

We can see that object 2436 is a **Class**. Its representation prints some of its fields (library and name), but we can access them all through its *data dictionary*:

In [6]:
s.refs[2436].x

{'predefined': False,
 'name': 'MyApp'->71516,
 'user_name': <base>null,
 'functions': Array(1)->63064,
 'functions_hash_table': <base>null,
 'fields': <base Array><empty_array>,
 'offset_in_words_to_field': <base>null,
 'interfaces': Array(0)->63063,
 'script': Script('package:myapp/main.dart')->11481,
 'library': Library('package:myapp/main.dart')->11847,
 'type_parameters': <base>null,
 'super_type': Type(Class('package:flutter/src/widgets/framework.dart', 'StatelessWidget')->1249)->48108,
 'signature_function': <base>null,
 'constants': <base Array><empty_array>,
 'declaration_type': Type(Class('package:myapp/main.dart', 'MyApp')->2436)->48663,
 'invocation_dispatcher_cache': <base Array><empty_array>,
 'allocation_stub': Code->18051,
 'cid': 981,
 'instance_size_in_words': 2,
 'next_field_offset_in_words': 2,
 'type_arguments_field_offset_in_words': -1,
 'num_type_arguments': 0,
 'num_native_fields': 64,
 'token_pos': 74,
 'end_token_pos': 954,
 'state_bits': 1097768}

As you can see some of the fields point to other objects, so we can go on and inspect them too:

In [7]:
s.refs[2436].x['super_type']

Type(Class('package:flutter/src/widgets/framework.dart', 'StatelessWidget')->1249)->48108

`snapshot.getrefs(...)` lists all objects of a certain type:

~~~ python
for obj in s.getrefs('Function'):
    # TODO
~~~

To iterate over all objects of a snapshot (including its base):

~~~ python
for ref in range(1, s.refs['next']):
    obj = s.refs[ref]
    # TODO
~~~

### Arrays

**Array** or **ImmutableArray** objects have one or more object items, which can be accessed through the `value` field in its data dictionary.  
However, it's better to just call `obj.values()`, which also handles the special case of an empty array:

In [11]:
obj = s.refs[2436].x['library'].x['dictionary']
print(obj)
print(obj.values())  # or obj.x['value']

Array(7)->63969
[<base>null, <base>null, Class('package:myapp/main.dart', '_MyHomePageState@465264790')->2061, Class('package:myapp/main.dart', 'MyApp')->2436, Function('main', 0)->9741, Class('package:myapp/main.dart', 'MyHomePage')->2099, Mint(4)->49841]


**GrowableObjectArray** objects have a *backing array* and a length:

In [9]:
s.refs[2436].x['library'].x['owned_scripts']

GrowableObjectArray(1, Array(3)->63967)->54880

`values()` works on these objects too:

In [10]:
s.refs[2436].x['library'].x['owned_scripts'].values()

[Script('package:myapp/main.dart')->11481]

### Strings

**OneByteString** and **TwoByteString** objects hold a string, which is printed in its representation:

In [13]:
s.refs[69306]

'Flutter Demo Home Page'->69306

You can access the string through the `value` field in its data dictionary:

In [15]:
s.refs[69306].x['value']

'Flutter Demo Home Page'

Snapshots have a `strings` dictionary, which allows you to access the object for a certain string:

In [16]:
s.strings['Flutter Demo Home Page']

'Flutter Demo Home Page'->69306

### Backreferences

So far, we've seen how an object references other objects through its data dictionary:

In [24]:
print(s.refs[2436])
print(s.refs[2436].x['script'])

Class('package:myapp/main.dart', 'MyApp')->2436
Script('package:myapp/main.dart')->11481


But we can look up *where is an object referenced from*, through its `src` attribute:

In [26]:
for backref in s.refs[2436].x['script'].src:
    print('Referenced from:', backref)

Referenced from: (Class('package:myapp/main.dart', '_MyHomePageState@465264790')->2061, 'script')
Referenced from: (Class('package:myapp/main.dart', 'MyHomePage')->2099, 'script')
Referenced from: (Class('package:myapp/main.dart', 'MyApp')->2436, 'script')
Referenced from: (Class('package:myapp/main.dart', '::')->2565, 'script')
Referenced from: (Array(3)->63967, 'value', 0)


Each item of `src` is a tuple; its first item is the referencing object, and the next item(s) describe the field where it's referenced.  
We can see that 4 classes reference the object with its `script` field, and an array contains the object in its first item.

This is pretty useful when combined with `snapshot.strings`. To find the `MyApp` class, we can get that string and look at its `src`:

In [33]:
s.strings['MyApp'].src

[(Class('package:myapp/main.dart', 'MyApp')->2436, 'name'),
 (Array(16386)->64169, 'value', 14917)]

And there it is, in the first tuple.

### Other operations

Use `obj.is_cid(...)` to check that an object is of certain kind, and `obj.ref` to get its reference number:

~~~ python
s.refs[2436].is_cid('Class')  # True
s.refs[2436].ref  # 2436
~~~

Calling `obj.describe()` returns its representation, plus (for code-related objects) some context describing its location in the code:

In [7]:
print(s.refs[18052])
print(s.refs[18052].describe())

Code->18052
Code->18052{ Function('build', 2)->9029 Class('package:myapp/main.dart', 'MyApp')->2436 }


### The root object

There's one more object, the root object, which doesn't have a ref number and can be accessed through `snapshot.refs['root']`. It has things like the global object pool, the symbol table, core classes, and some stubs (Code objects) for low-level tasks:

In [15]:
s.refs['root'].x

{'object_class': Class('Object')->1028,
 'object_type': Type(Class('Object')->1028)->48997,
 'null_class': Class('Null')->1027,
 'null_type': Type(Class('Null')->1027)->48996,
 'function_type': Type(Class('Function')->1029)->48995,
 'type_type': Type(Class('Type')->2591)->48994,
 'closure_class': Class('_Closure@0150898')->1026,
 'number_type': Type(Class('num')->1007)->48993,
 'int_type': Type(Class('int')->1832)->48992,
 'integer_implementation_class': Class('_IntegerImplementation@0150898')->1025,
 'int64_type': <base>null,
 'smi_class': Class('_Smi@0150898')->1024,
 'smi_type': Type(Class('_Smi@0150898')->1024)->48991,
 'mint_class': Class('_Mint@0150898')->1023,
 'mint_type': Type(Class('_Mint@0150898')->1023)->48990,
 'double_class': Class('_Double@0150898')->1022,
 'double_type': Type(Class('double')->2588)->48989,
 'float32x4_type': Type(Class('dart:typed_data', 'Float32x4')->2553)->48988,
 'int32x4_type': Type(Class('dart:typed_data', 'Int32x4')->2555)->48987,
 'float64x2_type

# Native analysis

If we look at the `Flutter Demo Home Page` string, the only objects that reference it are the `global_object_pool` and the `symbol_table`:

In [37]:
src = s.strings['Flutter Demo Home Page'].src
print(src)
assert src[0][0] == s.refs['root'].x['global_object_pool']
assert src[1][0] == s.refs['root'].x['symbol_table']

[(ObjectPool->19088, 'entries', 9188, 'raw_obj'), (Array(16386)->64169, 'value', 3064)]


It's used in the `MyApp.build` function, but since this reference is only in the assembled instructions, we don't know about it.  
That's what the `darter.asm` module is for. We use the `populate_native_references` method to analyze the instructions:

In [39]:
from darter.asm.base import populate_native_references
populate_native_references(s)

Starting analysis...
Done in 2.83s, processing results


And this populates an `nsrc` field on every object. This field tells us where it's used in native code:

In [40]:
s.strings['Flutter Demo Home Page'].nsrc

[(Code->18052, 2440608, 'load', 'ip')]

And if we look more closely at that `Code` object...

In [41]:
s.strings['Flutter Demo Home Page'].nsrc[0][0].describe()

"Code->18052{ Function('build', 2)->9029 Class('package:myapp/main.dart', 'MyApp')->2436 }"

We see it's the `build` function that we were expecting. The format of the tuples in `nsrc` is `(code object, instruction address, kind of reference, ...)`.